# NAND++ Configurations

_Version: 0.2_

This notebook contains code to evaluate NAND++ programs and view their configurations. When `DEBUG_MODE` is set to `1`, the configuration will be printed at every step of the execution.

At the moment the configuration  ignores additional extra variables that are obtained by the syntactic sugar transformations. 

### Ignore in first read: utility code

We use some utility code, which you can safely ignore in first read, to allow us to write NAND++ code in Python

In [1]:
# utility code 
%run "NAND programming language.ipynb"
from IPython.display import clear_output
clear_output()

In [2]:
DEBUG_MODE = 1

In [3]:
# Ignore this utility function in first and even second and third read
import inspect
import ast
import astor

def noop(f):
    return f;

def runwithstate(f):
    """Modify a function f to take and return an argument state and make all names relative to state."""
    tree = ast.parse(inspect.getsource(f))
    tmp = ast.parse("def _temp(state):\n    pass\n").body[0]
    args = tmp.args
    name = tmp.name
    tree.body[0].args = args
    tree.body[0].name = name
    tree.body[0].decorator_list = []
    

    class AddState(ast.NodeTransformer):
        def visit_Name(self, node: ast.Name):
            if node.id == "enandpp" or node.id == "nandpp" or node.id = "nandtm": 
                return ast.Name(id="noop", ctx=Load())
            new_node = ast.Attribute(ast.copy_location(ast.Name('state', ast.Load()), node), node.id,
                                     ast.copy_location(ast.Load(), node))
            return ast.copy_location(new_node, node)
        
    tree = AddState().visit(tree)
    tree.body[0].body = tree.body[0].body + [ast.parse("return state")]
    tree = ast.fix_missing_locations(tree)
    src = astor.to_source(tree)
    # print(src)
    exec(src,globals())
    _temp.original_func = f
    return _temp


In [4]:
# ignore utility class in first and even second or third read

from  collections import defaultdict
class NANDPPstate:
    """State of a NAND++ program."""
    
    def __init__(self):
        self.scalars = defaultdict(int)
        self.arrays  = defaultdict(lambda: defaultdict(int))
        self.i = 0
        # eventually should make self.i non-negative integer type
    
    def __setattr__(self,var,val):
        if var in ["i","scalars", "arrays"]:
            super().__setattr__(var,val)
            return
        if var[0].isupper():
            raise Exception
        else:
            self.scalars[var] = val
        
        
    def __getattr__(self,var):
        g =  globals()
        if var in g and callable(g[var]): return g[var]
        if var[0].isupper():
            if not var in self.arrays:
                self.arrays[var] = defaultdict(int)
            return self.arrays[var]
        else:
            if not var in self.scalars:
                self.scalars[var] = 0
            return self.scalars[var]
        
    def conf(self,i,imax):
        t = max([max([j for j in arr.keys()]) for arr in self.arrays.values() ])
        t = max(t,imax)
        out = ""
        l = max(len(s) for s in list(self.arrays.keys())+ list(self.scalars.keys()) )
        for arrname in sorted(list(self.arrays.keys())):
            line =blue(htmlspace(arrname.ljust(l+2)))
            for j in range(t+1):
                v = str((self.arrays[arrname])[j])
                s = green(v) if arrname in ["X","Xvalid"] else (red(v) if arrname in ["Y","Yvalid"] else v)
                line += bold(s) if i==j else s
            out += "<code>"+line + "</code><br>\n"
        for scalar in sorted(list(self.scalars.keys())):
            line = blue(htmlspace(scalar.ljust(l+2)))+"·"*i+bold(str(self.scalars[scalar]))+"·"*(t-i)
            out += "<code>"+line + "</code><br>\n"
        return out
            
        

In [5]:
from IPython.display import Markdown, display

def htmlspace(s):
    return s.replace(" ","&nbsp;")

def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string) if color else string
    display(Markdown(colorstr))
    
def color(c,s,j=0):
    return "<span style='color:{}'>{}</span>".format(c, s.ljust(j))
    
def bold(s,justify=0):
    return "**"+s.ljust(justify)+"**"
#  return "\x1b[1m"+s.ljust(justify)+"\x1b[0m"

#def underline(s,justify=0):
#    return "\x1b[4m"+s.ljust(justify)+"\x1b[24m"

def red(s,justify=0):
    return color("red",s,justify)  #.replace(" ","&nbsp;")
    #return  "\x1b[31m"+s.ljust(justify)+"\x1b[0m"


def green(s,justify=0):
    return color("green",s,justify) #.replace(" ","&nbsp;")
    # return  "\x1b[32m"+s.ljust(justify)+"\x1b[0m"


def blue(s,justify=0):
    return color("blue",s,justify) #.replace(" ","&nbsp;")
    # return  "\x1b[34m"+s.ljust(justify)+"\x1b[0m"

In [6]:
printmd(red(bold("&nbsp;a")+"b"))

<span style='color:None'><span style='color:red'>**&nbsp;a**b</span></span>

In [7]:
def index():
    """Generator for the values of i in the NAND++ sequence"""
    i = 0
    last = 0
    direction  = 1
    while True:
        yield i
        i += direction
        if i> last: 
            direction = -1
            last = i
        if i==0: direction = +1
            
a = index()
[next(a) for i in range(20)]

[0, 1, 0, 1, 2, 1, 0, 1, 2, 3, 2, 1, 0, 1, 2, 3, 4, 3, 2, 1]

In [8]:
import msvcrt  , os
def wait(): # change for non windows
    input("")
    
def NANDPPEVAL(f,X):
    """Evaluate a NAND++ function on input X"""
    s = NANDPPstate() # intialize state
    
    # copy input:
    for i in range(len(X)): 
        s.X[i] = X[i]
        s.Xvalid[i] = 1
        
    # main loop:
    t = 0
    step = 0
    if DEBUG_MODE:
        clear_output()
        printmd(f"__Starting configuration:__")
        printmd(s.conf(0,len(X)))
        wait()

    for  i in index(): 
        t = max(t,i,len(X))
        s.i = i
        s = f(s)
        if DEBUG_MODE:
            clear_output()
            printmd(f"__Iteration:__ {step},  `i`={i}")
            printmd(s.conf(i,t))
            wait()
        step += 1
        if not s.loop: break
    if DEBUG_MODE:
        print("Done!")
    
    # copy output:
    res = [] 
    i = 0
    while s.Yvalid[i]: 
        res += [s.Y[i]]
        i+= 1
    return res



def nandpp(f):
    """Modify python code to obtain NAND++ program"""
    g = runwithstate(f)
    def _temp1(X):
        return NANDPPEVAL(g,X)
    _temp1.original_func = f
    _temp1.transformed_func = g
    return _temp1

Here is the increment function in vanilla NAND++. Note that we need to keep track of an Array `Visited` to make sure we only add the carry once per location.

In [9]:
@nandpp
def inc():
    carry = IF(started,carry,one(started))
    started = one(started)
    Y[i] = IF(Visited[i],Y[i],XOR(X[i],carry))
    Visited[i] = one(started)
    carry = AND(X[i],carry)
    Yvalid[i] = one(started)
    loop = Xvalid[i]

In [ ]:
inc([1,1,1])

In [17]:
@nandpp
def inc():
    carry = IF(started,carry,one(started))
    started = one(started)
    Atstart[0] = one(started)
    idxincreasing = IF(Atstart[i],one(started),idxincreasing)
    idxincreasing = IF(Visited[i],zero(started),idxincreasing)
    Y[i] = IF(Visited[i],Y[i],XOR(X[i],carry))
    Visited[i] = one(started)
    carry = AND(X[i],carry)
    Yvalid[i] = one(started)
    loop = Xvalid[i]
    

In [18]:
inc([1,1,1,0,1])

<span style='color:None'>__Iteration:__ 25,  `i`=5</span>

<span style='color:None'><code><span style='color:blue'>Atstart&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>10000**0**</code><br>
<code><span style='color:blue'>Visited&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>11111**1**</code><br>
<code><span style='color:blue'>X&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><span style='color:green'>1</span><span style='color:green'>1</span><span style='color:green'>1</span><span style='color:green'>0</span><span style='color:green'>1</span>**<span style='color:green'>0</span>**</code><br>
<code><span style='color:blue'>Xvalid&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><span style='color:green'>1</span><span style='color:green'>1</span><span style='color:green'>1</span><span style='color:green'>1</span><span style='color:green'>1</span>**<span style='color:green'>0</span>**</code><br>
<code><span style='color:blue'>Y&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><span style='color:red'>0</span><span style='color:red'>0</span><span style='color:red'>0</span><span style='color:red'>1</span><span style='color:red'>1</span>**<span style='color:red'>0</span>**</code><br>
<code><span style='color:blue'>Yvalid&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span><span style='color:red'>1</span><span style='color:red'>1</span><span style='color:red'>1</span><span style='color:red'>1</span><span style='color:red'>1</span>**<span style='color:red'>1</span>**</code><br>
<code><span style='color:blue'>carry&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>·····**0**</code><br>
<code><span style='color:blue'>idxincreasing&nbsp;&nbsp;</span>·····**0**</code><br>
<code><span style='color:blue'>loop&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>·····**0**</code><br>
<code><span style='color:blue'>started&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>·····**1**</code><br>
</span>


Done!


[0, 0, 0, 1, 1, 0]

And here is the "vanilla NAND++" version of XOR:

In [ ]:
@nandpp
def vuXOR():
    Yvalid[0] = one(X[0])
    Y[0] = IF(Visited[i],Y[0],XOR(X[i],Y[0]))
    Visited[i] = one(X[0])
    loop = Xvalid[i]

In [ ]:
vuXOR([1,0,0,1,0,1])

In [12]:
def NANDPPcode(P):
    """Return NAND++ code of given function"""
    
    code = ''
    counter = 0
    
    
    class CodeNANDPPstate:
    
    
        def __getattribute__(self,var):
            # print(f"getting {var}")
            g =  globals()
            if var in g and callable(g[var]): return g[var]
            if var[0].isupper():  
                class Temp:
                    def __getitem__(self,k):  return var+"[i]"
                    def __setitem__(s,k,v):   
                        setattr(self,var+"[i]",v)            
                return Temp()
            return var
    
        def __init__(self):
            pass
    
        def __setattr__(self,var,val):
            nonlocal code
            # print(f"setting {var} to {val}")
            if code.find(val)==-1:
                code += f'\n{var} = {val}'
            else:
                code = rreplace(code,val,var)
    
    s = CodeNANDPPstate()
    
    def myNAND(a,b):
        nonlocal code, counter
        var = f'temp_{counter}'
        counter += 1
        code += f'\n{var} = NAND({a},{b})'
        return var
    
        
    
    
    
    s = runwith(lambda : P.transformed_func(s),"NAND",myNAND) 
    
    return code


# utility code - replace string from right, taken from stackoverflow
def rreplace(s, old, new, occurrence=1): 
    li = s.rsplit(old, occurrence)
    return new.join(li)



In [13]:
print(NANDPPcode(inc))


temp_0 = NAND(started,started)
temp_1 = NAND(started,temp_0)
temp_2 = NAND(started,started)
temp_3 = NAND(temp_1,temp_2)
temp_4 = NAND(carry,started)
carry = NAND(temp_3,temp_4)
temp_6 = NAND(started,started)
started = NAND(started,temp_6)
Atstart[i] = NAND(started,started)
temp_9 = NAND(started,started)
temp_10 = NAND(started,temp_9)
temp_11 = NAND(Atstart[i],Atstart[i])
temp_12 = NAND(indexincreasing,temp_11)
temp_13 = NAND(temp_10,Atstart[i])
idxincreasing = NAND(temp_12,temp_13)
temp_15 = NAND(started,started)
temp_16 = NAND(started,temp_15)
temp_17 = NAND(temp_16,temp_16)
temp_18 = NAND(Visited[i],Visited[i])
temp_19 = NAND(indexincreasing,temp_18)
temp_20 = NAND(temp_17,Visited[i])
idxincreasing = NAND(temp_19,temp_20)
temp_22 = NAND(X[i],carry)
temp_23 = NAND(X[i],temp_22)
temp_24 = NAND(carry,temp_22)
temp_25 = NAND(temp_23,temp_24)
temp_26 = NAND(Visited[i],Visited[i])
temp_27 = NAND(temp_25,temp_26)
temp_28 = NAND(Y[i],Visited[i])
Y[i] = NAND(temp_27,temp_28)
temp_30 = NAND(

In [ ]:
@nandpp
def myinc():
    temp_0 = NAND(started,started)
    temp_1 = NAND(started,temp_0)
    temp_2 = NAND(started,started)
    temp_3 = NAND(temp_1,temp_2)
    temp_4 = NAND(carry,started)
    carry = NAND(temp_3,temp_4)
    temp_6 = NAND(started,started)
    started = NAND(started,temp_6)
    Atstart[i] = NAND(started,started)
    temp_9 = NAND(started,started)
    temp_10 = NAND(started,temp_9)
    temp_11 = NAND(Atstart[i],Atstart[i])
    temp_12 = NAND(indexincreasing,temp_11)
    temp_13 = NAND(temp_10,Atstart[i])
    idxincreasing = NAND(temp_12,temp_13)
temp_15 = NAND(started,started)
temp_16 = NAND(started,temp_15)
temp_17 = NAND(temp_16,temp_16)
temp_18 = NAND(Visited[i],Visited[i])
temp_19 = NAND(indexincreasing,temp_18)
temp_20 = NAND(temp_17,Visited[i])
idxincreasing = NAND(temp_19,temp_20)
temp_22 = NAND(X[i],carry)
temp_23 = NAND(X[i],temp_22)
temp_24 = NAND(carry,temp_22)
temp_25 = NAND(temp_23,temp_24)
temp_26 = NAND(Visited[i],Visited[i])
temp_27 = NAND(temp_25,temp_26)
temp_28 = NAND(Y[i],Visited[i])
Y[i] = NAND(temp_27,temp_28)
temp_30 = NAND(started,started)
Visited[i] = NAND(started,temp_30)
temp_32 = NAND(X[i],carry)
carry = NAND(temp_32,temp_32)
temp_34 = NAND(started,started)
Yvalid[i] = NAND(started,temp_34)
loop = Xvalid[i]